In [ ]:
import os

import pandas as pd
import numpy as np
np.set_printoptions(precision=6, suppress=True)

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import *
tf.__version__

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.dates as mdates
from matplotlib.ticker import (LinearLocator, MultipleLocator, FormatStrFormatter)
from matplotlib.dates import MONDAY
from matplotlib.dates import MonthLocator, WeekdayLocator, DateFormatter
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = ((8/2.54), (6/2.54))
plt.rcParams["font.family"] = "Arial"
plt.rcParams["mathtext.default"] = "rm"
plt.rcParams.update({'font.size': 11})
MARKER_SIZE = 15
cmap_m = ["#f4a6ad", "#f6957e", "#fccfa2", "#8de7be", "#86d6f2", "#24a9e4", "#b586e0", "#d7f293"]
cmap = ["#e94d5b", "#ef4d28", "#f9a54f", "#25b575", "#1bb1e7", "#1477a2", "#a662e5", "#c2f442"]

plt.rcParams['axes.spines.top'] = False
# plt.rcParams['axes.edgecolor'] = 
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.width'] = 1

In [ ]:
DATA_INTV = '10 min'
target_df = pd.read_csv('./results/SW2_greenhouse.csv', index_col='Unnamed: 0')
target_df = target_df[['temp', 'hum', 'rad', 'loadcell_1', 'loadcell_2']]
target_df.index = pd.DatetimeIndex(target_df.index)
target_df = target_df.resample(DATA_INTV).first()
target_df = target_df.interpolate(limit=35)
target_df = target_df.dropna()

In [ ]:
_p = target_df.index[0] - pd.Timedelta(DATA_INTV)
index_start = target_df.index[0]
data_indices = []
for _ in target_df.index:
    if (_ - _p) != pd.Timedelta(DATA_INTV):
        data_indices.append([index_start, _p])
        index_start = _
        print(_ - _p)
    _p = _
data_indices.append([index_start, _p])

In [ ]:
data_indices = [
    [pd.Timestamp('2020-03-05 00:00:00'), pd.Timestamp('2020-04-20 16:47:00')],
    [pd.Timestamp('2020-06-01 00:00:00'), pd.Timestamp('2020-06-15 23:59:00')],
    [pd.Timestamp('2020-05-16 00:00:00'), pd.Timestamp('2020-05-31 23:59:00')],
    [pd.Timestamp('2020-05-01 00:00:00'), pd.Timestamp('2020-05-15 23:59:00')],
    [pd.Timestamp('2020-04-23 00:00:00'), pd.Timestamp('2020-04-30 23:59:00')],
    [pd.Timestamp('2020-06-16 00:00:00'), pd.Timestamp('2020-07-03 23:59:00')]
]

In [ ]:
# fig = plt.figure(figsize=((8/2.54*6.2), (6/2.54*3.2)))
# ax0 = plt.subplot()

# ax0.spines['right'].set_visible(False)
# ax0.spines['left'].set_position(('outward', 5))
# ax0.spines['bottom'].set_position(('outward', 5))

# ax0.plot(target_df.index, target_df['loadcell_1']/4, c=cmap[3])
# ax0.plot(target_df.index, target_df['loadcell_2']/4, c=cmap[0])
# ax0.plot(target_df.index, target_df['loadcell_3']/4, c=cmap[4])

# ax0.set_xbound(target_df.index.min(), target_df.index.max())
# # ax0.set_xbound(pd.Timestamp('2020-04-20 16:15:00') + pd.Timedelta('1h 24 min'), pd.Timestamp('2020-04-20 17:43:00') + pd.Timedelta('1h 24 min'))
# ax0.xaxis.set_major_locator(LinearLocator(20))
# ax0.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

# ax0.yaxis.set_major_locator(LinearLocator(5))
# ax0.yaxis.set_minor_locator(LinearLocator(13))
# ax0.set_ybound(2, 5)

# ax0.set_xlabel('Date')
# ax0.set_ylabel('Weight (kg)')

# ax0.set_rasterized(True)
# fig.tight_layout()
# plt.show() 

In [ ]:
MAXS = target_df.max().values
MINS = target_df.min().values
SCREEN_SIZE = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EPSILON = 1e-05
BEST_PATH = './checkpoints/UNet_loadcell_%d' % SCREEN_SIZE

In [ ]:
# data_indices = shuffle(data_indices, random_state=930217)

In [ ]:
data_indices

In [ ]:
temp_current = []
temp_prev = []
temp_next = []
temp_mask = []
temp_label = []
for _ in data_indices[:3]:
    print(_)
    temp_df = target_df[_[0]:_[1]].values
    if temp_df.shape[0] < SCREEN_SIZE*3:
        print('Shorter than the screen size.')
        continue
    np.random.seed(3101)
    null_prob = 0.2
    mask = np.random.choice(2, temp_df.size, p=[null_prob, 1-null_prob]).reshape(temp_df.shape)
    np.random.seed(4564)
    raw_null_prob = 0.3
    null_len = 60
    raw_indices = np.random.choice(np.arange(int(temp_df.shape[0]/null_len)-1),
                                   replace=False, size=int(temp_df.shape[0]/null_len * raw_null_prob))
    raw_indices = raw_indices*null_len
    raw_indices_ext = []
    if len(raw_indices) == 0:
        print('Too short.')
        continue
    for elem in raw_indices:
        for _ in range(elem, elem+null_len):
            raw_indices_ext.append(_)
    raw_indices = np.array(raw_indices_ext)
    raw_indices = np.unique(raw_indices[raw_indices < temp_df.shape[0]])
    temp_df = (temp_df - MINS)/(MAXS - MINS)
    
    mask[raw_indices, :] = 0
    missing_df = np.ma.array(temp_df, mask=1-mask, fill_value=-1)
    missing_df = missing_df.filled()
    for INDEX in range(SCREEN_SIZE*2, temp_df.shape[0]-SCREEN_SIZE):
        temp_prev.append(missing_df[(INDEX-SCREEN_SIZE*2):(INDEX-SCREEN_SIZE), :])
        temp_current.append(missing_df[(INDEX-SCREEN_SIZE):INDEX, :])
        temp_next.append(missing_df[INDEX:INDEX+SCREEN_SIZE, :])
        
        temp_mask.append(mask[(INDEX-SCREEN_SIZE):INDEX, :])
        temp_label.append(temp_df[(INDEX-SCREEN_SIZE):INDEX, :])
temp_prev = np.stack(temp_prev).astype(np.float32)
temp_current = np.stack(temp_current).astype(np.float32)
temp_next = np.stack(temp_next).astype(np.float32)
temp_mask = np.stack(temp_mask).astype(np.float32)
temp_label = np.stack(temp_label).astype(np.float32)

raw_input = np.stack([temp_current, temp_mask, temp_prev, temp_next], axis = -1)
raw_label = temp_label[..., np.newaxis]

In [ ]:
del(temp_current)
del(temp_prev)
del(temp_next)
del(temp_mask)
del(temp_label)

In [ ]:
raw_input = np.concatenate([raw_input]*(int(SCREEN_SIZE/target_df.shape[1])), axis=2)
raw_label = np.concatenate([raw_label]*(int(SCREEN_SIZE/target_df.shape[1])), axis=2)

In [ ]:
temp_current = []
temp_prev = []
temp_next = []
temp_mask = []
temp_label = []
for _ in data_indices[3:]:
    print(_)
    temp_df = target_df[_[0]:_[1]].values
    if temp_df.shape[0] < SCREEN_SIZE*3:
        print('Shorter than the screen size.')
        continue
    np.random.seed(3101)
    null_prob = 0.2
    mask = np.random.choice(2, temp_df.size, p=[null_prob, 1-null_prob]).reshape(temp_df.shape)
    np.random.seed(4564)
    raw_null_prob = 0.3
    null_len = 60
    raw_indices = np.random.choice(np.arange(int(temp_df.shape[0]/null_len)-1),
                                   replace=False, size=int(temp_df.shape[0]/null_len * raw_null_prob))
    raw_indices = raw_indices*null_len
    raw_indices_ext = []
    if len(raw_indices) == 0:
        print('Too short.')
        continue
    for elem in raw_indices:
        for _ in range(elem, elem+null_len):
            raw_indices_ext.append(_)
    raw_indices = np.array(raw_indices_ext)
    raw_indices = np.unique(raw_indices[raw_indices < temp_df.shape[0]])
    temp_df = (temp_df - MINS)/(MAXS - MINS)
    
    mask[raw_indices, :] = 0
    missing_df = np.ma.array(temp_df, mask=1-mask, fill_value=-1)
    missing_df = missing_df.filled()
    for INDEX in range(SCREEN_SIZE*2, temp_df.shape[0]-SCREEN_SIZE):
        temp_prev.append(missing_df[(INDEX-SCREEN_SIZE*2):(INDEX-SCREEN_SIZE), :])
        temp_current.append(missing_df[(INDEX-SCREEN_SIZE):INDEX, :])
        temp_next.append(missing_df[INDEX:INDEX+SCREEN_SIZE, :])
        
        temp_mask.append(mask[(INDEX-SCREEN_SIZE):INDEX, :])
        temp_label.append(temp_df[(INDEX-SCREEN_SIZE):INDEX, :])
temp_prev = np.stack(temp_prev).astype(np.float32)
temp_current = np.stack(temp_current).astype(np.float32)
temp_next = np.stack(temp_next).astype(np.float32)
temp_mask = np.stack(temp_mask).astype(np.float32)
temp_label = np.stack(temp_label).astype(np.float32)

test_input = np.stack([temp_current, temp_mask, temp_prev, temp_next], axis = -1)
test_label = temp_label[..., np.newaxis]

In [ ]:
test_input = np.concatenate([test_input]*(int(SCREEN_SIZE/target_df.shape[1])), axis=2)
test_label = np.concatenate([test_label]*(int(SCREEN_SIZE/target_df.shape[1])), axis=2)

In [ ]:
del(temp_current)
del(temp_prev)
del(temp_next)
del(temp_mask)
del(temp_label)

In [ ]:
print(raw_input.shape)
print(raw_label.shape)

In [ ]:
print(test_input.shape)
print(test_label.shape)

In [ ]:
i = 5
print(raw_input[i, ..., 2])
print(raw_input[i, ..., 0])
print(raw_input[i, ..., 3])
print(raw_input[i, ..., 1])
print(raw_label[i, ..., 0])

In [ ]:
print(MAXS)
print(MINS)

In [ ]:
# raw_input = raw_input.astype(np.float32)
# raw_label = raw_label.astype(np.float32)
# test_input = test_input.astype(np.float32)
# test_label = test_label.astype(np.float32)

In [ ]:
raw_input, raw_label = shuffle(raw_input, raw_label, random_state=4574)
test_input, test_label = shuffle(test_input, test_label, random_state=3110)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((raw_input, raw_label))
train_dataset = train_dataset.cache().shuffle(BATCH_SIZE*50).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((test_input, test_label))
val_dataset = val_dataset.cache().shuffle(BATCH_SIZE*50).batch(BATCH_SIZE)

In [ ]:
class ConvBlock(layers.Layer):
    def __init__(self, filters, kernel_size):
        super(ConvBlock, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        
        self.conv1 = layers.Conv2D(self.filters, self.kernel_size, kernel_initializer='he_normal', padding='same')
        self.batch1 = layers.BatchNormalization()
        self.activation1 = layers.Activation(tf.nn.leaky_relu)
        self.conv2 = layers.Conv2D(self.filters, self.kernel_size, kernel_initializer='he_normal', padding='same')
        self.batch2 = layers.BatchNormalization()
        self.activation2 = layers.Activation(tf.nn.leaky_relu)
        
    def call(self, inp, TRAINING):
        
        inp = self.activation1(self.batch1(self.conv1(inp), training=TRAINING))
        inp = self.activation2(self.batch2(self.conv2(inp), training=TRAINING))
        
        return inp

In [ ]:
class DeconvBlock(layers.Layer):
    def __init__(self, filters, kernel_size, strides):
        super(DeconvBlock, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        
        self.deconv1 = layers.Conv2DTranspose(self.filters, self.kernel_size, kernel_initializer='he_normal',
                                              strides=self.strides, padding='same')
        self.activation1 = layers.Activation(tf.nn.leaky_relu)
        
    def call(self, inp):
        
        inp = self.activation1(self.deconv1(inp))
        
        return inp

In [ ]:
class UNet(Model):
    def __init__(self):
        super(UNet, self).__init__()
        
        self.conv_block1 = ConvBlock(64, (2, 2))
        self.pool1 = layers.MaxPooling2D()
        
        self.conv_block2 = ConvBlock(128, (2, 2))
        self.pool2 = layers.MaxPooling2D()
        
        self.conv_block3 = ConvBlock(256, (2, 2))
        self.pool3 = layers.MaxPooling2D()
        
        self.conv_block4 = ConvBlock(512, (2, 2))
        self.pool4 = layers.MaxPooling2D()
        
        self.conv_bottom = ConvBlock(1024, (2, 2))
        
        self.deconv_block1 = DeconvBlock(512, (2, 2), (2, 2))
        self.conv_block_r1 = ConvBlock(512, (2, 2))
        
        self.deconv_block2 = DeconvBlock(256, (2, 2), (2, 2))
        self.conv_block_r2 = ConvBlock(256, (2, 2))
        
        self.deconv_block3 = DeconvBlock(128, (2, 2), (2, 2))
        self.conv_block_r3 = ConvBlock(128, (2, 2))
        
        self.deconv_block4 = DeconvBlock(64, (2, 2), (2, 2))
        self.conv_block_r4 = ConvBlock(64, (2, 2))
        
        self.padding = layers.ZeroPadding2D(((1, 0), (0, 1)))
        self.output_conv = layers.Conv2D(1, (1, 1), activation='sigmoid')
        
    def call(self, inp, TRAINING=False):
        
        conv1 = self.conv_block1(inp, TRAINING)
        pooled1 = self.pool1(conv1)
        conv2 = self.conv_block2(pooled1, TRAINING)
        pooled2 = self.pool2(conv2)
        conv3 = self.conv_block3(pooled2, TRAINING)
        pooled3 = self.pool3(conv3)
        conv4 = self.conv_block4(pooled3, TRAINING)
        pooled4 = self.pool4(conv4)
        
        bottom = self.conv_bottom(pooled4, TRAINING)
        
        deconv1 = self.deconv_block1(bottom)
        deconv1 = layers.concatenate([deconv1, conv4])
        deconv1 = self.conv_block_r1(deconv1, TRAINING)
        deconv2 = self.deconv_block2(deconv1)
        deconv2 = layers.concatenate([deconv2, conv3])
        deconv2 = self.conv_block_r2(deconv2, TRAINING)
        deconv3 = self.padding(self.deconv_block3(deconv2))
        deconv3 = layers.concatenate([deconv3, conv2])
        deconv3 = self.conv_block_r3(deconv3, TRAINING)
        deconv4 = self.deconv_block4(deconv3)
        deconv4 = layers.concatenate([deconv4, conv1])
        deconv4 = self.conv_block_r4(deconv4, TRAINING)
        
        return self.output_conv(deconv4)

In [ ]:
#loss inputs should be masked.
loss_object = tf.keras.losses.MeanSquaredError()
def loss_function(model, inp, tar, TRAINING=False):
    
    masked_real = tar * (1 - inp[..., 1:2])
    masked_pred = model(inp, TRAINING) * (1 - inp[..., 1:2])
    
    return loss_object(masked_real, masked_pred)

In [ ]:
# loss_object = tf.keras.losses.MeanSquaredError()
# def loss_function(model, inp, tar, TRAINING=False):
    
#     masked_real = tar
#     masked_pred = model(inp, TRAINING)
    
#     return loss_object(masked_real, masked_pred)

In [ ]:
unet_model = UNet()
opt = tf.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON)

In [ ]:
@tf.function
def train(loss_function, model, opt, inp, tar):
    with tf.GradientTape() as tape:
        gradients = tape.gradient(loss_function(model, inp, tar, True), model.trainable_variables)
        gradient_variables = zip(gradients, model.trainable_variables)
        opt.apply_gradients(gradient_variables)

In [ ]:
checkpoint_path = BEST_PATH

ckpt = tf.train.Checkpoint(unet_model=unet_model,
                           opt=opt)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=10)

In [ ]:
writer = tf.summary.create_file_writer('tmp')

In [ ]:
prev_test_loss = 100.0
early_stop_buffer = 100
DISP_STEPS = 100
with writer.as_default():
    with tf.summary.record_if(True):
        for epoch in range(500):
            for step, (inp, tar) in enumerate(train_dataset):
                train(loss_function, unet_model, opt, inp, tar)
                loss_values = loss_function(unet_model, inp, tar)
                tf.summary.scalar('loss', loss_values, step=step)
                
                if step % DISP_STEPS == 0:
                    test_loss = 0
                    for step_, (inp_, tar_) in enumerate(val_dataset):
                        test_loss += loss_function(unet_model, inp_, tar_)
                        
                        if step_ > DISP_STEPS:
                            test_loss /= DISP_STEPS
                            break
                    if test_loss.numpy() < prev_test_loss:
                        ckpt_save_path = ckpt_manager.save()
                        prev_test_loss = test_loss.numpy()
                        print('Saving checkpoint at {}'.format(ckpt_save_path))
                        early_stop_buffer = 100
                    else:
                        early_stop_buffer -= 1
                    print('Epoch {} batch {} train loss: {:.4f} test loss: {:.4f}'
                          .format(epoch, step, loss_values.numpy(), test_loss.numpy()))
                if early_stop_buffer <= 0:
                    print('early stop.')
                    break
            if early_stop_buffer <= 0:
                    break                    

In [ ]:
i = -1
if ckpt_manager.checkpoints:
    ckpt.restore(ckpt_manager.checkpoints[i])
    print ('Checkpoint ' + ckpt_manager.checkpoints[i][-2:] +' restored!!')

In [ ]:
pred_result = unet_model.predict(val_dataset)

In [ ]:
avg_pred = []
OUTLIER = 3
for __ in range(target_df.shape[1]):
    temp = []
    for _ in range(int(pred_result.shape[1]/target_df.shape[1])):
        temp.append(pred_result[..., _*target_df.shape[1]:(_+1)*target_df.shape[1], 0][..., __])
    temp = np.stack(temp, axis=2)
    temp.sort(axis=2)
    avg_pred.append(temp[..., OUTLIER:-OUTLIER].mean(axis=2))
avg_pred = np.stack(avg_pred, axis=2)

In [ ]:
masking = test_input[..., 1]
avg_masking = masking[..., :target_df.shape[1]]

In [ ]:
masked_pred = np.ma.array(pred_result[..., 0], mask=masking)
masked_avg_pred = np.ma.array(avg_pred, mask=avg_masking)
masked_label = np.ma.array(test_label[..., 0], mask=masking)

In [ ]:
target_df.columns

In [ ]:
x_t = np.arange(0, test_label.shape[1])
for _ in range (2):

        NUMBERS = np.arange(1, pred_result.shape[0])
        np.random.shuffle(NUMBERS)
        NUMBERS = NUMBERS[:6]
        position = 331
        fig = plt.figure(figsize=((8.5/2.54)*8, (6/2.54)*8))
        
        i=-1
        for NUMBER in NUMBERS:
            ax = plt.subplot(position)
            measured1 = plt.plot(x_t, test_label[NUMBER, :, i], c='k', alpha=1) #measured
            expect1 = plt.plot(x_t, masked_pred[NUMBER, :, i], 'o', c=cmap[5], alpha=1) #estimated
            expect1 = plt.plot(x_t, masked_avg_pred[NUMBER, :, i], 'o', c=cmap[0], alpha=1) #estimated
            expect2 = plt.plot(x_t, pred_result[NUMBER, :, i], c=cmap[2], alpha=1) #estimated
            ax.axis('off')

            position += 1
        plt.show()
        _ += 1